## INTRODUCTION

The project was developed on Jupyter Notebook.

In this notebook, several features of a dataset containing about 130,000 wine reviews have been explored. The libraries used are exclusively those shown in the course of Data Manipulation and Visualization. 

The exploration of the dataset consists of 5 steps: Setup, Data Understanding, Data Cleaning, Preliminary Analysis and Deep Feature Analysis. Matplotlib and Seaborn charts were used in the preliminary analyses, while to have more complete and detailed graphs, I chose Plotly.

For proper use, run all cells from top to bottom. Currently the graphs are not displayed to not overburden the file and facilitate its loading. Each note will populate the file with specific graphs. It is recommended to use the "Restart Kernel and Run All Cells" command from the "Run" section of the command bar.

## Step 0: Setup

Importing libraries

In [ ]:
import pandas as pd # Importing pandas
import matplotlib.pyplot as plt # Importing matplotlib
import seaborn as sns # Importing seaborn
import plotly # Importing plotly
import plotly.express as px # Importing px, a simpler graphical interface
import plotly.graph_objects as go # Importing go, a more advanced graphical interface than px
import numpy as np # Importing numpy
pd.set_option('display.max_columns', 200) # Allows display in head() up to 200 columns
import warnings
warnings.filterwarnings('ignore') # Ignore some error messages that indicate obsolete methods in use

I proceed with reading the CSV file downloaded from Kaggle and uploaded to a Google Drive link viewable by anyone who has the link

https://drive.google.com/file/d/15wbDCriZiIoGVIo-zZKgj8-w9m36umU9/view?usp=sharing

In [ ]:
# Creating URL with file ID
file_id = '15wbDCriZiIoGVIo-zZKgj8-w9m36umU9'
url = f"https://drive.google.com/uc?id={file_id}"

# Creating the dataframe by reading the URL
df = pd.read_csv(url)


## Step 1: Data Understanding

In [ ]:
df.head() # Display the first 5 lines

In [ ]:
df.tail() # Display the last 5 lines

In [ ]:
df.shape # Display number of rows and columns of the dataframe

In [ ]:
df.columns # Display the name of each column

In [ ]:
df.dtypes # Display the data types present in the dataframe

In [ ]:
df.describe() # Display descriptive statistics

In [ ]:
df.isnull().sum() # Display quantity of null values per column

## Step 2: Data Cleaning

I start by deleting the "Unnamed" column. This represents a mere numerical index already present in the data visualization of Jupyter Notebook, is therefore redundant.


In [ ]:
df.drop(columns=["Unnamed: 0"], inplace=True)

I replace the missing NaN values in the "price" column.

I will insert the median value for consistency with its asymetric distribution.


In [ ]:
df["price"].fillna(df["price"].median(), inplace=True)

#inplace=True to make changes directly on the dataframe

I then replace the null values of the columns "region_1", "region_2", "taster_name", "taster_twitter_handle", "designation".

I put "Unknown" as they are strings and can not enter useful values like the median for an int or a float

In [ ]:
df["designation"].fillna("Unknown", inplace=True)
df["taster_name"].fillna("Unknown", inplace=True)
df["taster_twitter_handle"].fillna("Unknown", inplace=True)
df["region_1"].fillna("Unknown", inplace=True)
df["region_2"].fillna("Unknown", inplace=True)

Finally I proceed with the elimination of the rows (there are few) with NaN values in crucial columns like "country", "provinces" and "variety".

In [ ]:
df.dropna(subset=["country", "province", "variety"], inplace=True)

In [ ]:
df.isnull().sum() # I display the amount of null values again

In [ ]:
df.duplicated() # Ok, there are no duplicates

## Step 3: Preliminary Analysis

To begin to understand the dataset I decide to explore in general the trend of prices and score, the different varieties and the origin of wines. I used charts generated with matplotlib library and/or seaborn for this preliminary phase.

In [ ]:
df.head()

In [ ]:
 print(df[['points', 'price']].describe())

I immediately notice a much wider range of values on the price, found at first sight not only by the quartiles but also by the narrow standard deviation on scores and wide on prices.

It is plausible to think that the prices of wines present outliers upwards (given the rather low average) that move the Gaussian distribution to the right. I check it with a histogram.

In [ ]:
plt.figure(figsize=(12,5)) #Set chart size
sns.histplot(df['price'], bins=100, kde=True, color='purple') # bins to set the width of the bars, kde to draw the distribution line
plt.title('Distribution of wine prices') # title
plt.xlabel('Prices') # name of axis x
plt.ylabel('Number of wines') # name of axis y
plt.show()


As expected the distribution is concentrated around values <100 and contains outliers up to 3300 which alter the average values. For this kind of data, it is more effective to work on the median. I will go deeper with more advanced charts in the next step.

Now for the scores:

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=df['points'], color='orange')
plt.title('Distribution of wine scores')
plt.xlabel('Points')
plt.show()


Scores are spread over values ranging from 80 to 98, with exceptions to 99 and 100.

Therefore I assume that this dataset only concerns high quality wines and that I will be able to make a more specific analysis later on about what they are and if there are correlations with their price.

I’m now running an analysis on the most reviewed countries

In [ ]:
top_countries = df['country'].value_counts().head(5) # Create a list that counts the top 5 countries present

plt.figure(figsize=(10,4))
sns.barplot(x=top_countries.values, y=top_countries.index, palette="viridis") # Sulle x inserisco il conteggio (value), sulle y i paesi (key)
plt.title('Top 5 Countries with more reviews')
plt.xlabel('Number of reviews')
plt.ylabel('Country')
plt.show()


The United States is the country with the largest number of wines on the list. It is not certain that they are the most appreciated: this quantitative analysis can be integrated with a qualitative one to observe the correlation between number of reviews and value of the scores.

Using a pie chart, I analyse the different varieties:

In [ ]:
top_varieties = df['variety'].value_counts().head(10) # I create a list that counts the top 10 varieties present

plt.figure(figsize=(7,7)) 
plt.pie(top_varieties,
        labels=top_varieties.index,  # Labels with the names of the varieties
        autopct='%1.1f%%',  # Show the percentages with only one decimal place
        colors=sns.color_palette("Set3"),
        startangle=180,  # Rotate the chart 180° for an aesthetic choice
        wedgeprops={'edgecolor': 'black'}  # Insert black border to separate the slices
)

plt.title("Top 10 wine varieties by number of reviews")
plt.show()

Now that we have the percentage distribution of the varieties, it may be interesting to see in which areas they are produced for the most part or which wineries have a monopoly on a certain variety.


## Step 4: Deep Feature Analysis

I deepen the preliminary analyses carried out previously going to integrate in graphs generated with Plotly, to have greater completeness of data and the possibility of interacting with the graph at points of interest.

In [ ]:
# 1. Distribution of prices

In [ ]:
fig = px.histogram(df, x='price', 
                   nbins=500, # Density of each bar
                   color_discrete_sequence=['orange'], 
                   marginal='box', # I generate a box above the chart that acts as a scatter to interact with each wine
                   hover_data=['title', 'variety', 'country'] # Allows you to view the output at mouse-over
)

fig.update_layout(title='Distribution of wine prices', 
                  xaxis_title='Price', 
                  yaxis_title='Frequence')

# Draw midline
fig.add_shape(type='line', 
              x0=df['price'].median(), x1=df['price'].median(), 
              y0=0, y1=40000, 
              line=dict(color='black', dash='dash', width = 3))

fig.show()

It is now possible to zoom the graph in the area of greater concentration, but at the same time be able to correctly display the outliers that move the average to the right. The median value plotted in black is positioned on the area of higher concentration going to verify the hypothesis of greater reliability.

In [ ]:
# 2. Distribution of scores

In [ ]:
import re

# Extraction of the vintage from the "title" column
def extract_year(title):
    years = re.findall(r'\b(19[0-9]{2}|20[0-2][0-9])\b', title) #Find all the years from 1900 to 2029 in the title
    return int(years[0]) if years else None # Returns the first year found as integer, otherwise None

df["year"] = df["title"].apply(extract_year) # Apply the function to the "title" column and create a new "year" column
df_year = df.dropna(subset=["year"]) # Remove the lines where "year" is NaN


top_years = df_year["year"].value_counts().nlargest(10).index # Select only the 10 most reviewed vintages
df_year_filtered = df_year[df_year["year"].isin(top_years)].sort_values("year") # Filter the df to include only these 10 years, and I order them


# Creation of the chart
fig = px.scatter(
    df_year_filtered, x="price", y="points", 
    facet_col="year",  # Divide the charts by year
    hover_data=["winery","title"],  
    title="Distribuzione Punteggio-Prezzo per le 10 annate più rappresentate",
    labels={"price": "Prezzo", "points": "Punteggio", "year": "Anno", "winery": "Cantina", "title": "Vino"},
    color_discrete_sequence=["purple"],
    opacity=0.5
)

# I remove the symbol "=" in facet_columns and transform the values "year" into whole numbers
fig.for_each_annotation(lambda a: a.update(text=str(int(float(a.text.split("=")[-1])))))

fig.show()

This scatter plot shows a strong presence of wines with score >=90 and at the same time shows a not excessive correlation with the price. We see that, for example, the most expensive wine on the whole list, produced in 2013, is well below other wines much cheaper but with putteggi also equal to 100. By analysing a heatmap of the correlation between these two parameters:

In [ ]:
df_corr = df[['points','price']].dropna().corr()
sns.heatmap(df_corr, annot=True)
plt.show()

It is clear that, although present with a value of 0.4, the correlation between price and score increase is very slight and not always consequential.

In [ ]:
# 3. Scatter reviews vs average score by country

In [ ]:
df_country = df.groupby('country') \
               .agg({'points': 'mean', 'country': 'size'}) \
               .rename(columns={'country': 'count'}) \
               .reset_index()

fig = px.scatter(df_country, x='count', y='points', 
                 size='count',
                 size_max = 100,
                 color='points', 
                 hover_name='country', 
                 color_continuous_scale='Viridis')

fig.update_layout(title='Correlation between reviews and average scores by country', 
                  xaxis_title='Number of reviews', 
                  yaxis_title='Average score')

fig.show()

This chart shows substantial differences between the quantity of reviews and the quality of the latter: although the USA is by far the most present country on the list, its wines have lower scores than those of other 6 minor countries. Austria is the country with the highest average score and the only one to reach a value of 90 points, despite the reduced presence of wines in the catalog compared to USA, France, Italy. It is interesting to see how countries with very few wines in the catalogue, such as Hungary, Canada, Switzerland or Luxembourg, have very high scores confirming the relationship between quantity of reviews and product quality.

Another interesting chart could be a multi-layered pie chart that shows the distribution by price first for countries and then more specifically for provinces. It would be possible to insert an additional third layer with also the region of interest, but the chart becomes very dense and illegible.

In [ ]:
fig = px.sunburst(df, path = ['country', 'province'], values = 'price',
                 title = 'Distribution prices by state and province')
fig.show()

In [ ]:
# 4. Distribution of varieties

In [ ]:
# Filter data to exclude any "unknown" provinces
df_filtered = df[df['province'] != 'Unknown']

# Calculation of the number of varieties by province
region_variety_counts = df_filtered.groupby(['province', 'variety']).size().reset_index(name='counts')

# Calculation of total wines by province
region_counts = df_filtered['province'].value_counts().reset_index()
region_counts.columns = ['province', 'total_counts']

# Selection and filter of the top 10 provinces
top_10_regions = region_counts.head(10)['province']
region_variety_counts_top_10 = region_variety_counts[region_variety_counts['province'].isin(top_10_regions)]

fig = px.bar(region_variety_counts_top_10, x='province', y='counts', color='variety', 
              title='Distribution of varieties for the top 10 provinces',
              labels={'variety': 'Variety','province': 'Province', 'counts': 'Number of Wines'})
fig.show()

Globally, California is the province that produces the most wines, more than three times as many as the first contender. The predominant varieties previously observed, Pinot Noir and Chardonnay, represent the two main varieties produced in California. The production of Chardonnay californian alone exceeds the entire wine list of Tuscany, Bordeaux and Oregon, provinces in the top 5 world producers. Along with the province of Washington, California is also proving to be one of the most versatile in terms of variety.

I now observe the relationship between variety and average score:

In [ ]:
# Calculate the average score for each variety
variety_scores = df.groupby('variety')['points'].mean().reset_index()
variety_scores.columns = ['variety', 'average_points']

# Sort by average score
variety_scores = variety_scores.sort_values("average_points")

fig = px.line(variety_scores, x='variety', y='average_points', template='seaborn',
              labels={'variety': 'Varietà', 'average_points': 'Punteggio Medio'},
              title='Average score for wine varieties',
              hover_data={'average_points': True})
fig.show()

The line chart confirms average scores above 80 but maximum score 95, therefore no winery can produce only wines with score = 100. By interacting with the graph, you can see the average score for each variety.

I will conclude by observing which wineries are the most important in terms of average score and the respective varieties produced:

In [ ]:
# Calculate the average score for each winery
winery_scores = df.groupby('winery')['points'].mean().reset_index()
winery_scores.columns = ['winery', 'average_points']

# I select the 5 best
top_5_wineries = winery_scores.nlargest(5, 'average_points')['winery']

# Filter the original df to include only the top 5
df_top_5_wineries = df[df['winery'].isin(top_5_wineries)]

# Calculate the percentage of varieties of wines produced for these wineries
variety_counts = df_top_5_wineries.groupby(['winery', 'variety']).size().reset_index(name='counts') # How many wines of each variety are produced by each winery
total_counts = df_top_5_wineries.groupby('winery').size().reset_index(name='total_counts') # How many total wines each winery produces
variety_counts = pd.merge(variety_counts, total_counts, on='winery') # Combine the two dataframes 
variety_counts['percentage'] = (variety_counts['counts'] / variety_counts['total_counts']) * 100 # Calculate the percentage 

# I add the average scores to df variety_counts
variety_counts = pd.merge(variety_counts, winery_scores, on='winery')

fig = px.bar(variety_counts, x='average_points', y='winery', color='variety', 
             title='Punteggio medio delle 5 migliori cantine & rispettive varietà',
             labels={'winery': 'Cantina', 'average_points': 'Punteggio Medio Totale', 'variety': 'Varietà'},
             )

fig.update_layout(
    bargap=0.2,       
    bargroupgap=0.3,  
    width=1100,
    height=500
)

fig.show()

It can be seen that the 5 best wineries all have very high average scores and similar to each other even though the Araujo winery is in first place for 1 point. Each of these 5 wineries specialises in a single wine variety. Cabernet Sauvignon is a variety produced by both the first and second of the list.